In [5]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

def make_episode_url(main_url, no, week='wed'):
    parsed = urlparse(main_url)
    query = parse_qs(parsed.query)
    titleId = query.get('titleId', [None])[0]
    if titleId is None:
        raise ValueError("main_url에 titleId가 없습니다.")

    new_query = {
        'titleId': titleId,
        'no': no,
        'week': week
    }
    query_string = urlencode(new_query)
    detail_url = urlunparse((
        parsed.scheme,
        parsed.netloc,
        '/webtoon/detail',
        '',
        query_string,
        ''
    ))
    return detail_url

def download_one_episode(title, no, url):
    # 1. list url에서 detail url 생성
    episode_url = make_episode_url(url, no)
    print(f"[INFO] 다운로드 대상 회차 URL: {episode_url}")

    req_header = {
        'referer': episode_url,
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
    }

    res = requests.get(episode_url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        # 이미지 태그 추출
        img_tags = soup.select("div.wt_viewer img[src^='https://image-comic.pstatic.net/webtoon']")
        print(f"[INFO] 이미지 개수: {len(img_tags)}")

        img_url_list = [img['src'] for img in img_tags]

        # 저장할 폴더 생성 (img/제목/회차번호)
        imgdir_name = os.path.join('img', title, str(no))
        os.makedirs(imgdir_name, exist_ok=True)

        for img_url in img_url_list:
            res1 = requests.get(img_url, headers=req_header)
            if res1.ok:
                img_data = res1.content
                file_name = os.path.basename(img_url.split('?')[0])  # 쿼리 제거
                file_path = os.path.join(imgdir_name, file_name)
                with open(file_path, 'wb') as file:
                    print(f"[저장] {file_path} ({len(img_data):,} byte)")
                    file.write(img_data)
            else:
                print(f"[에러] 이미지 다운로드 실패: {img_url} 코드 {res1.status_code}")
    else:
        print(f"[에러] 회차 페이지 요청 실패: {episode_url} 코드 {res.status_code}")

# 예시 실행
download_one_episode('칼부림', '2', 'https://comic.naver.com/webtoon/list?titleId=602916')

[INFO] 다운로드 대상 회차 URL: https://comic.naver.com/webtoon/detail?titleId=602916&no=2&week=wed
[INFO] 이미지 개수: 12
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_1.jpg (658,497 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_2.jpg (316,788 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_3.jpg (357,045 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_4.jpg (283,124 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_5.jpg (510,832 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_6.jpg (401,560 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_7.jpg (555,482 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_8.jpg (516,041 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_9.jpg (449,490 byte)
[저장] img\칼부림\2\20131211154915_8b940ff9d5992b2134343d75d9ffa6fb_IMAG01_10